## Huggingface login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Axolotl 설치

In [ ]:
import torch
# Check so there is a gpu available, a T4(free tier) is enough to run this notebook
assert (torch.cuda.is_available()==True)

In [ ]:
!pip install torch=="2.1.2"
!pip install -e git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
!pip install flash-attn=="2.5.0"
!pip install deepspeed=="0.13.1"!pip install mlflow=="2.13.0"

## Custom Dataset 추가

In [ ]:
%mkdir data
!wget -O ./data/ko_civil_service-multiturn.json https://github.com/superdom/blog/raw/main/062024-axolotl/ko_civil_service-multiturn.json
%ls ./data | grep ko_civil_service-multiturn.json

## Training 환경 구성

In [ ]:
import yaml

yaml_string = """
base_model: meta-llama/Meta-Llama-3-8B-Instruct
model_type: LlamaForCausalLM
tokenizer_type: AutoTokenizer

load_in_8bit: false
load_in_4bit: true
strict: false

chat_template: llama3
datasets:
  - path: data/ko_civil_service-multiturn.json
    type: sharegpt
    chat_template: llama3

dataset_prepared_path:
output_dir: ./outputs/llama-3-8b-Instruct-multiturn-bnb-4bit-qlora/multiturn

sequence_len: 4096
sample_packing: false
pad_to_sequence_len: true

adapter: qlora
lora_model_dir:
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_linear: true
lora_fan_in_fan_out:

#wandb_project:
#wandb_entity:
#wandb_watch:
#wandb_name:
#wandb_log_model:

gradient_accumulation_steps: 2
micro_batch_size: 1
num_epochs: 5
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: false
fp16: true
tf32: false

gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention: false
flash_attention: false

warmup_steps: 5
evals_per_epoch: 1
eval_table_size:
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
  pad_token: <|end_of_text|>
  eos_token: <|eot_id|>
"""

yaml_dict = yaml.safe_load(yaml_string)

file_path = 'llama-3-8b-Instruct-bnb-4bit-qlora.yaml'

with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

## Training!

In [ ]:
!accelerate launch -m axolotl.cli.train /content/llama-3-8b-Instruct-bnb-4bit-qlora.yaml

## Gradio로 Inference

In [ ]:
!accelerate launch -m axolotl.cli.inference /content/llama-3-8b-Instruct-bnb-4bit-qlora.yaml \
    --qlora_model_dir="./outputs/llama-3-8b-Instruct-multiturn-bnb-4bit-qlora/multiturn" --gradio